<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/99_GenerarVisualizacionesPDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Sistema de Generación de PDFs
===================================================================

Script completo listo para ejecutar en Google Colab que incluye:
- Montaje automático de Google Drive
- Instalación de dependencias
- Detección automática de PDFs existentes
- Sistema de checkpoints
- Trazabilidad completa con TQDM

Autor: Jesús L.
Universidad: Universidad Oberta de Catalunya (UOC)

"""

# =============================================================================
# CONFIGURACIÓN INICIAL
# =============================================================================

print("=" * 70)
print("SISTEMA DE GENERACIÓN DE PDFs - TFM")
print("=" * 70)
print("\nPaso 1/4: Montando Google Drive...")

from google.colab import drive
drive.mount('/content/drive', force_remount=False)

print("✓ Google Drive montado correctamente\n")

# =============================================================================
# INSTALACIÓN DE DEPENDENCIAS
# =============================================================================

print("Paso 2/4: Instalando dependencias...")
import subprocess
import sys

resultado = subprocess.run(
    [sys.executable, "-m", "pip", "install", "-q", "img2pdf", "Pillow", "tqdm"],
    capture_output=True
)

if resultado.returncode == 0:
    print("✓ Dependencias instaladas: img2pdf, Pillow, tqdm\n")
else:
    print("⚠ Error instalando dependencias")
    print(resultado.stderr.decode())

SISTEMA DE GENERACIÓN DE PDFs - TFM

Paso 1/4: Montando Google Drive...
Mounted at /content/drive
✓ Google Drive montado correctamente

Paso 2/4: Instalando dependencias...
✓ Dependencias instaladas: img2pdf, Pillow, tqdm



In [ ]:
# =============================================================================
# CÓDIGO PRINCIPAL
# =============================================================================

print("Paso 3/4: Cargando módulos...\n")

import json
import logging
import re
import pickle
from collections import defaultdict
from pathlib import Path
from typing import Dict, List, Set, Tuple, Optional

try:
    import img2pdf
    from tqdm.auto import tqdm
except ImportError as e:
    print(f"ERROR: No se pudieron importar dependencias: {e}")
    print("Ejecute: !pip install img2pdf Pillow tqdm")
    raise

# Configuración de logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

Paso 3/4: Cargando módulos...



In [ ]:

class AlphaWarningCounter(logging.Handler):
    """Handler para contar warnings de alpha channel de img2pdf."""

    def __init__(self):
        super().__init__()
        self.count = 0

    def emit(self, record):
        if 'alpha' in record.getMessage().lower():
            self.count += 1


class GeneradorPDFVisualizaciones:
    """
    Generador de PDFs con detección automática de progreso y checkpoints.

    Características:
    - Detecta PDFs ya existentes (no los regenera)
    - Guarda checkpoint cada 5 PDFs
    - Continúa automáticamente si se interrumpe
    - Trazabilidad completa con TQDM
    """

    PATRON_UMBRAL_DECIMAL = re.compile(r'_t0[._]\d+$')
    PATRON_SENSIBILIDAD_ESPECIFICA = re.compile(
        r'_(baja|media|alta|maxima)_sensibilidad_t0[._]\d+$'
    )
    EXTENSIONES_VALIDAS = {'.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG'}
    CHECKPOINT_INTERVALO = 5

    def __init__(self, ruta_indice: str, directorio_salida: str) -> None:
        """Inicializa el generador con detección de PDFs existentes."""
        self.ruta_indice = Path(ruta_indice)
        self.directorio_salida = Path(directorio_salida)
        self.checkpoint_file = self.directorio_salida / '.checkpoint.pkl'

        if not self.ruta_indice.exists():
            raise FileNotFoundError(f"Índice maestro no encontrado: {self.ruta_indice}")

        self.directorio_salida.mkdir(parents=True, exist_ok=True)

        self.indice_maestro: Optional[Dict] = None
        self.agrupaciones: defaultdict = defaultdict(list)

        self.estadisticas = {
            'visualizaciones_omitidas': 0,
            'archivos_no_encontrados': 0,
            'extensiones_invalidas': 0,
            'alpha_warnings': 0,
            'pdfs_generados': 0,
            'errores': 0,
            'total_imagenes_procesadas': 0
        }

        # Configurar contador de alpha warnings
        self.alpha_counter = AlphaWarningCounter()
        self.alpha_counter.setLevel(logging.WARNING)
        img2pdf_logger = logging.getLogger('img2pdf')
        img2pdf_logger.addHandler(self.alpha_counter)

        # Cargar checkpoint O detectar PDFs existentes
        self.configs_completadas = self._cargar_o_detectar_progreso()

        logger.info("Inicializado GeneradorPDFVisualizaciones v1.2.1")
        logger.info(f"  Índice: {self.ruta_indice}")
        logger.info(f"  Salida: {self.directorio_salida}")
        if self.configs_completadas:
            logger.info(f"  PDFs detectados: {len(self.configs_completadas)}")

    def _cargar_o_detectar_progreso(self) -> Set[str]:
        """
        Carga checkpoint O detecta PDFs existentes en el directorio.

        Permite continuar sin perder progreso de ejecuciones previas.
        """
        # Intentar cargar checkpoint
        if self.checkpoint_file.exists():
            try:
                with open(self.checkpoint_file, 'rb') as f:
                    checkpoint_data = pickle.load(f)
                    if isinstance(checkpoint_data, set):
                        logger.info("✓ Checkpoint encontrado")
                        return checkpoint_data
            except Exception as e:
                logger.warning(f"Checkpoint corrupto: {e}")

        # Si no hay checkpoint, detectar PDFs existentes
        logger.info("Detectando PDFs existentes en directorio...")
        pdfs_existentes = set()

        try:
            for pdf_file in self.directorio_salida.glob('*.pdf'):
                config_name = pdf_file.stem
                pdfs_existentes.add(config_name)

            if pdfs_existentes:
                logger.info(f"✓ Detectados {len(pdfs_existentes)} PDFs existentes")
                logger.info("  Estos NO serán regenerados")
                self._guardar_checkpoint_inicial(pdfs_existentes)
            else:
                logger.info("  No se encontraron PDFs previos")

            return pdfs_existentes

        except Exception as e:
            logger.warning(f"Error detectando PDFs: {e}")
            return set()

    def _guardar_checkpoint_inicial(self, configs: Set[str]) -> None:
        """Guarda checkpoint inicial con PDFs detectados."""
        try:
            with open(self.checkpoint_file, 'wb') as f:
                pickle.dump(configs, f, protocol=pickle.HIGHEST_PROTOCOL)
            logger.info("✓ Checkpoint inicial creado")
        except Exception as e:
            logger.warning(f"No se pudo crear checkpoint: {e}")

    def _guardar_checkpoint(self) -> None:
        """Guarda checkpoint periódicamente."""
        try:
            with open(self.checkpoint_file, 'wb') as f:
                pickle.dump(self.configs_completadas, f, protocol=pickle.HIGHEST_PROTOCOL)
        except Exception as e:
            logger.warning(f"Error guardando checkpoint: {e}")

    def _eliminar_checkpoint(self) -> None:
        """Elimina checkpoint al finalizar."""
        if self.checkpoint_file.exists():
            try:
                self.checkpoint_file.unlink()
                logger.info("✓ Checkpoint eliminado (proceso completado)")
            except Exception as e:
                logger.warning(f"Error eliminando checkpoint: {e}")

    def cargar_indice(self) -> None:
        """Carga el índice maestro desde JSON."""
        logger.info("Cargando índice maestro...")
        with open(self.ruta_indice, 'r', encoding='utf-8') as archivo:
            self.indice_maestro = json.load(archivo)
        logger.info(f"✓ Índice cargado: {len(self.indice_maestro)} fotografías")

    def normalizar_codigo_config(self, codigo_config: str) -> str:
        """Normaliza códigos de configuración eliminando umbrales."""
        match_sensibilidad = self.PATRON_SENSIBILIDAD_ESPECIFICA.search(codigo_config)
        if match_sensibilidad:
            return self.PATRON_UMBRAL_DECIMAL.sub('', codigo_config)
        match_umbral = self.PATRON_UMBRAL_DECIMAL.search(codigo_config)
        if match_umbral:
            return self.PATRON_UMBRAL_DECIMAL.sub('', codigo_config)
        return codigo_config

    def extraer_configuraciones(self) -> Set[str]:
        """Extrae y valida configuraciones del índice con barra de progreso."""
        logger.info("Extrayendo configuraciones...")
        configuraciones_encontradas: Set[str] = set()
        visualizaciones_procesadas = 0

        pbar_fotos = tqdm(
            self.indice_maestro.items(),
            desc="Escaneando fotografías",
            unit="foto",
            total=len(self.indice_maestro),
            ncols=100
        )

        for foto_id, datos_foto in pbar_fotos:
            modelos_disponibles = datos_foto.get('modelos_disponibles', {})

            for codigo_config, info_modelo in modelos_disponibles.items():
                ruta_viz = info_modelo.get('ruta_visualizacion')

                if not ruta_viz:
                    self.estadisticas['visualizaciones_omitidas'] += 1
                    continue

                ruta_viz_path = Path(ruta_viz)
                if not ruta_viz_path.exists():
                    self.estadisticas['archivos_no_encontrados'] += 1
                    continue

                if ruta_viz_path.suffix not in self.EXTENSIONES_VALIDAS:
                    self.estadisticas['extensiones_invalidas'] += 1
                    continue

                config_normalizado = self.normalizar_codigo_config(codigo_config)
                self.agrupaciones[config_normalizado].append((foto_id, str(ruta_viz_path)))
                configuraciones_encontradas.add(config_normalizado)
                visualizaciones_procesadas += 1

            pbar_fotos.set_postfix({
                'configs': len(configuraciones_encontradas),
                'viz': visualizaciones_procesadas
            })

        pbar_fotos.close()

        logger.info(f"✓ Configuraciones: {len(configuraciones_encontradas)}")
        logger.info(f"✓ Visualizaciones válidas: {visualizaciones_procesadas}")

        if self.estadisticas['visualizaciones_omitidas'] > 0:
            logger.warning(f"⚠ Sin ruta: {self.estadisticas['visualizaciones_omitidas']}")

        return configuraciones_encontradas

    def ordenar_visualizaciones(self, visualizaciones: List[Tuple[str, str]]) -> List[str]:
        """Ordena visualizaciones alfabéticamente por foto_id."""
        visualizaciones_ordenadas = sorted(visualizaciones, key=lambda tupla: tupla[0])
        return [ruta for _, ruta in visualizaciones_ordenadas]

    def generar_pdf(
        self,
        config_normalizado: str,
        rutas_imagenes: List[str],
        pbar: Optional[tqdm] = None
    ) -> Path:
        """Genera un PDF consolidado a partir de imágenes."""
        nombre_pdf = f"{config_normalizado}.pdf"
        ruta_pdf = self.directorio_salida / nombre_pdf

        if pbar:
            pbar.set_description(f"PDF: {nombre_pdf[:45]}")

        try:
            count_before = self.alpha_counter.count
            pdf_bytes = img2pdf.convert(rutas_imagenes)
            count_after = self.alpha_counter.count

            alpha_warnings_este_pdf = count_after - count_before
            if alpha_warnings_este_pdf > 0:
                self.estadisticas['alpha_warnings'] += alpha_warnings_este_pdf

            with open(ruta_pdf, 'wb') as archivo_pdf:
                archivo_pdf.write(pdf_bytes)

            tamanio_mb = ruta_pdf.stat().st_size / (1024 * 1024)
            self.estadisticas['pdfs_generados'] += 1
            self.estadisticas['total_imagenes_procesadas'] += len(rutas_imagenes)

            if pbar:
                pbar.set_postfix({
                    'MB': f'{tamanio_mb:.1f}',
                    'imgs': len(rutas_imagenes),
                    'alpha': self.estadisticas['alpha_warnings']
                })

            return ruta_pdf

        except Exception as error:
            self.estadisticas['errores'] += 1
            logger.error(f"Error en {nombre_pdf}: {error}")
            raise

    def generar_todos_los_pdfs(self, forzar_reinicio: bool = False) -> Dict[str, Path]:
        """
        Genera todos los PDFs pendientes con checkpoints.

        Args:
            forzar_reinicio: Si True, ignora PDFs existentes y empieza desde cero
        """
        print("\n" + "=" * 70)
        print("GENERACIÓN DE PDFs CON AUTO-DETECCIÓN Y CHECKPOINTS")
        print("=" * 70 + "\n")

        if forzar_reinicio:
            if self.checkpoint_file.exists():
                self._eliminar_checkpoint()
            self.configs_completadas = set()
            logger.info("⚠ Reinicio forzado: ignorando PDFs existentes")

        self.cargar_indice()
        configuraciones = self.extraer_configuraciones()

        if not configuraciones:
            logger.error("✗ No se encontraron configuraciones")
            return {}

        configs_pendientes = sorted(configuraciones - self.configs_completadas)

        if not configs_pendientes:
            print("\n" + "=" * 70)
            print("¡TODAS LAS CONFIGURACIONES YA ESTÁN COMPLETAS!")
            print("=" * 70)
            print(f"\nTotal: {len(configuraciones)} PDFs")
            print(f"Ubicación: {self.directorio_salida}")
            self._eliminar_checkpoint()
            return {}

        print(f"CONFIGURACIÓN DEL PROCESO:")
        print(f"  Total configuraciones:     {len(configuraciones):>4}")
        print(f"  Ya completadas (previas):  {len(self.configs_completadas):>4}")
        print(f"  Pendientes por generar:    {len(configs_pendientes):>4}")
        print(f"  Checkpoint cada:           {self.CHECKPOINT_INTERVALO:>4} PDFs")
        print()

        pdfs_generados: Dict[str, Path] = {}
        errores: List[Tuple[str, str]] = []

        pbar_pdfs = tqdm(
            configs_pendientes,
            desc="Generando PDFs",
            unit="PDF",
            ncols=100,
            colour='green',
            initial=len(self.configs_completadas),
            total=len(configuraciones)
        )

        contador_desde_checkpoint = 0

        for config in pbar_pdfs:
            try:
                visualizaciones = self.agrupaciones[config]
                rutas_ordenadas = self.ordenar_visualizaciones(visualizaciones)

                ruta_pdf = self.generar_pdf(config, rutas_ordenadas, pbar=pbar_pdfs)
                pdfs_generados[config] = ruta_pdf
                self.configs_completadas.add(config)

                contador_desde_checkpoint += 1

                if contador_desde_checkpoint >= self.CHECKPOINT_INTERVALO:
                    self._guardar_checkpoint()
                    contador_desde_checkpoint = 0

            except Exception as error:
                errores.append((config, str(error)))
                continue

        pbar_pdfs.close()

        if contador_desde_checkpoint > 0:
            self._guardar_checkpoint()

        if len(self.configs_completadas) >= len(configuraciones):
            self._eliminar_checkpoint()

        self._mostrar_resumen(pdfs_generados, errores, len(configuraciones))

        return pdfs_generados

    def _mostrar_resumen(
        self,
        pdfs_generados: Dict[str, Path],
        errores: List[Tuple[str, str]],
        total_configs: int
    ) -> None:
        """Muestra resumen detallado del proceso."""
        print("\n" + "=" * 70)
        print("RESUMEN DE GENERACIÓN")
        print("=" * 70 + "\n")

        print("ESTADÍSTICAS:")
        print("-" * 70)
        print(f"  Total configuraciones:           {total_configs:>6}")
        print(f"  PDFs ya existentes (previos):    {len(self.configs_completadas) - len(pdfs_generados):>6}")
        print(f"  PDFs generados esta sesión:      {len(pdfs_generados):>6}")
        print(f"  PDFs totales completados:        {len(self.configs_completadas):>6}")
        print(f"  Imágenes procesadas (sesión):    {self.estadisticas['total_imagenes_procesadas']:>6}")
        print(f"  Errores durante generación:      {len(errores):>6}")
        print(f"  Warnings de canal alpha:         {self.estadisticas['alpha_warnings']:>6}")

        if pdfs_generados:
            tamanio_total_mb = sum(
                pdf.stat().st_size for pdf in pdfs_generados.values()
            ) / (1024 * 1024)
            print(f"  Tamaño generado (sesión):        {tamanio_total_mb:>6.2f} MB")

        print("-" * 70)

        if errores:
            print(f"\nERRORES DETECTADOS ({len(errores)}):")
            for config, error in errores[:5]:
                print(f"  - {config}: {error[:60]}...")
            if len(errores) > 5:
                print(f"  ... y {len(errores) - 5} errores más")

        if self.estadisticas['alpha_warnings'] > 0:
            print(f"\nNOTA: {self.estadisticas['alpha_warnings']} imágenes con canal alpha procesadas.")
            print("      (Normal en PNGs con transparencia)")

        pendientes = total_configs - len(self.configs_completadas)
        if pendientes > 0:
            print(f"\n⚠ QUEDAN {pendientes} CONFIGURACIONES PENDIENTES")
            print("  Ejecute nuevamente esta celda para continuar")
        else:
            print("\n✓ ¡PROCESO COMPLETADO EXITOSAMENTE!")
            print(f"  Ubicación: {self.directorio_salida}")

        print("\n" + "=" * 70 + "\n")


# =============================================================================
# CELDA 4: EJECUCIÓN PRINCIPAL
# =============================================================================

def main(forzar_reinicio: bool = False) -> None:
    """
    Función principal para ejecutar la generación.

    Args:
        forzar_reinicio: Si True, ignora checkpoint y PDFs existentes
    """
    print("Paso 4/4: Iniciando generación de PDFs...\n")

    # Configuración de rutas del TFM
    RUTA_INDICE = "/content/drive/MyDrive/TFM/3_Analisis/fase1_integracion/indice_maestro.json"
    RUTA_SALIDA = "/content/drive/MyDrive/TFM/3_Analisis/00_Visualizaciones"

    try:
        generador = GeneradorPDFVisualizaciones(
            ruta_indice=RUTA_INDICE,
            directorio_salida=RUTA_SALIDA
        )

        pdfs_generados = generador.generar_todos_los_pdfs(forzar_reinicio=forzar_reinicio)

        if pdfs_generados:
            print(f"✓ Sesión completada: {len(pdfs_generados)} PDFs nuevos generados\n")

    except FileNotFoundError as error:
        print(f"\n✗ ERROR: Archivo no encontrado")
        print(f"  {error}")
        print(f"\nVerifique que las rutas sean correctas:")
        print(f"  Índice: {RUTA_INDICE}")
        print(f"  Salida: {RUTA_SALIDA}")
        raise

    except Exception as error:
        print(f"\n✗ ERROR INESPERADO: {error}")
        logger.exception("Traceback completo:")
        raise


# =============================================================================
# EJECUCIÓN
# =============================================================================

if __name__ == "__main__":
    # Ejecutar generación
    # Para reiniciar desde cero: main(forzar_reinicio=True)
    main()